<a href="https://colab.research.google.com/github/Mohammed-khair/AI_Game_Jam_EXPAND/blob/master/Convolutional_neural_network_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building a Convolutional neural network from scratch

This project was done as an excercise in the following course [Coursera: Convolutional neural network](https://www.coursera.org/learn/convolutional-neural-networks)

In this project, we will build:


*   Convolutional Operation
*   Pooling Operation
*   CNN
*   CNN Forward Propagation



We will only use the [numpy](https://numpy.org/) Library

In [ ]:
import numpy as np
import h5py
import matplotlib.pyplot as plt

np.random.seed(1)

# 1 Padding

First we will create a helper function to do padding

What is Padding?
Padding refers to adding extra border pixels around the input data before applying convolutional operations



Why Padding?
Padding help in:
*   **Avoiding the shrinking problem:** Without padding, the feature map size would decrease with each convolutional layer. This shrinking effect can lead to a loss of spatial resolution, and the network might not be able to capture fine-grained patterns.
*   **Controlling the output size:** By adjusting the amount of padding, it is possible to control the size of the output feature maps in CNNs, which can be useful in certain architectural designs and when building specific network structures.
*  **Retaining information from the input borders:** Padding ensures that the convolution operation can be applied to all elements of the input, allowing information from the borders to be captured in the output.

To implement the function we will use [np.pad](https://docs.scipy.org/doc/numpy/reference/generated/numpy.pad.html)

In [ ]:
def zero_pad(X, pad):
    """
    Pad with zeros all images of the dataset X. The padding is applied to the height and width of an image,
    as illustrated in Figure 1.

    Argument:
    X -- python numpy array of shape (m, n_H, n_W, n_C) representing a batch of m images
    pad -- integer, amount of padding around each image on vertical and horizontal dimensions

    Returns:
    X_pad -- padded image of shape (m, n_H + 2 * pad, n_W + 2 * pad, n_C)
    """

    pad_width = ((0, 0), (pad, pad), (pad, pad), (0, 0))
    X_pad = np.pad(X, pad_width, mode='constant', constant_values=0)

    return X_pad

# 2 Convolution Step

Next, we  will implement a single step of convolution, corresponding to applying a filter to just one of the positions to get a single real-valued output. \
Here is what a single step of convolution looks [like](https://www.researchgate.net/figure/Schematic-illustration-of-a-convolutional-operation-The-convolutional-kernel-shifts-over_fig2_332190148)

In [ ]:
def conv_single_step(a_slice_prev, W, b):
    """
    Apply one filter defined by parameters W on a single slice (a_slice_prev) of the output activation
    of the previous layer.

    Arguments:
    a_slice_prev -- slice of input data of shape (f, f, n_C_prev)
    W -- Weight parameters contained in a window - matrix of shape (f, f, n_C_prev)
    b -- Bias parameters contained in a window - matrix of shape (1, 1, 1)

    Returns:
    Z -- a scalar value, the result of convolving the sliding window (W, b) on a slice x of the input data
    """

    s = np.multiply(a_slice_prev,  W)  # Element-wise product between a_slice_prev and W
    Z = np.sum(s)             # Sum over all entries of the volume s
    Z += float(b)               # Add bias b to Z. Cast b to a float() so that Z results in a scalar value.

    return Z

# 3  Forward Prop

Now we will create a function to convolve the filter ***W*** on an an input activation ***A_prev***

**Reminder**:
    
The formulas relating the output shape of the convolution to the input shape are:
    
$$n_H = \Bigl\lfloor \frac{n_{H_{prev}} - f + 2 \times pad}{stride} \Bigr\rfloor +1$$
$$n_W = \Bigl\lfloor \frac{n_{W_{prev}} - f + 2 \times pad}{stride} \Bigr\rfloor +1$$
$$n_C = \text{number of filters used in the convolution}$$

In [ ]:
def conv_forward(A_prev, W, b, hparameters):
    """
    Implements the forward propagation for a convolution function

    Arguments:
    A_prev -- output activations of the previous layer,
        numpy array of shape (m, n_H_prev, n_W_prev, n_C_prev)
    W -- Weights, numpy array of shape (f, f, n_C_prev, n_C)
    b -- Biases, numpy array of shape (1, 1, 1, n_C)
    hparameters -- python dictionary containing "stride" and "pad"

    Returns:
    Z -- conv output, numpy array of shape (m, n_H, n_W, n_C)
    cache -- cache of values needed for the conv_backward() function
    """

    # Retrieve dimensions from A_prev's shape
    (m, n_H_prev, n_W_prev, n_C_prev) = A_prev.shape

    # Retrieve dimensions from W's shape
    (f, f, n_C_prev, n_C) = W.shape

    # Retrieve information from "hparameters"
    stride = hparameters["stride"]
    pad = hparameters["pad"]

    # Compute the dimensions of the CONV output volume using the formula given above.
    # int() is used to do the floor operation
    n_H = int( (n_H_prev + 2 * pad - f) / stride) + 1
    n_W = int( (n_W_prev + 2 * pad - f) / stride) + 1

    # Initialize the output volume Z with zeros.
    Z = np.zeros((m, n_H, n_W, n_C))

    # Create A_prev_pad by padding A_prev
    A_prev_pad = zero_pad(A_prev, pad)

    # loop over the batch of training examples
    for i in range(m):

      # Select ith training example's padded activation
        a_prev_pad = A_prev_pad[i]

        # loop over vertical axis of the output volume
        for h in range(n_H):

            # Find the vertical start and end of the current "slice"
            vert_start = h * stride
            vert_end = vert_start + f

            # loop over horizontal axis of the output volume
            for w in range(n_W):

                # Find the horizontal start and end of the current "slice"
                horiz_start = w * stride
                horiz_end = horiz_start + f

                # loop over channels (= #filters) of the output volume
                for c in range(n_C):

                    # Use the corners to define the (3D) slice of a_prev_pad (See Hint above the cell)
                    a_slice_prev = a_prev_pad[vert_start:vert_end, horiz_start:horiz_end, :]

                    # Convolve the (3D) slice with the correct filter W and bias b, to get back one output neuron.
                    weights = W[:, :, :, c]
                    biases = b[:, :, :, c]
                    Z[i, h, w, c] = conv_single_step(a_slice_prev, weights, biases)

    # Save information in "cache" for the backprop
    cache = (A_prev, W, b, hparameters)

    return Z, cache

# 4 Pooling Layer

The pooling layer reduces the height and width of the input. It helps reduce computation, as well as helps make feature detectors more invariant to its position in the input. The two types of pooling layers are

In [ ]:
def pool_forward(A_prev, hparameters, mode = "max"):
    """
    Implements the forward pass of the pooling layer

    Arguments:
    A_prev -- Input data, numpy array of shape (m, n_H_prev, n_W_prev, n_C_prev)
    hparameters -- python dictionary containing "f" and "stride"
    mode -- the pooling mode you would like to use, defined as a string ("max" or "average")

    Returns:
    A -- output of the pool layer, a numpy array of shape (m, n_H, n_W, n_C)
    cache -- cache used in the backward pass of the pooling layer, contains the input and hparameters
    """

    # Retrieve dimensions from the input shape
    (m, n_H_prev, n_W_prev, n_C_prev) = A_prev.shape

    # Retrieve hyperparameters from "hparameters"
    f = hparameters["f"]
    stride = hparameters["stride"]

    # Define the dimensions of the output
    n_H = int(1 + (n_H_prev - f) / stride)
    n_W = int(1 + (n_W_prev - f) / stride)
    n_C = n_C_prev

    # Initialize output matrix A
    A = np.zeros((m, n_H, n_W, n_C))

    # loop over the training examples
    for i in range(m):

          # loop on the vertical axis of the output volume
         for h in range(n_H):

            # Find the vertical start and end of the current "slice" (≈2 lines)
            vert_start = h * stride
            vert_end = vert_start + f

            # loop on the horizontal axis of the output volume
            for w in range(n_W):

                # Find the vertical start and end of the current "slice" (≈2 lines)
                horiz_start = w * stride
                horiz_end = horiz_start + f

                # loop over the channels of the output volume
                for c in range (n_C):

                     # Use the corners to define the current slice on the ith training example of A_prev, channel c. (≈1 line)
                    a_prev_slice = A_prev[i, vert_start:vert_end, horiz_start:horiz_end, c]

                    # Compute the pooling operation on the slice.
                    if mode == "max":
                        A[i, h, w, c] = np.max(a_prev_slice)
                    elif mode == "average":
                        A[i, h, w, c] = np.mean(a_prev_slice)

    # Store the input and hparameters in "cache" for pool_backward()
    cache = (A_prev, hparameters)

    # Making sure your output shape is correct
    assert(A.shape == (m, n_H, n_W, n_C))

    return A, cache